In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import datasets


import mlflow
from mlflow.models import infer_signature # set the schema of the model


In [27]:
# set the tracking URI 
mlflow.set_tracking_uri("http://localhost:5000")


In [28]:
# Load the iris dataset
X,y = datasets.load_iris(return_X_y=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Define the model hyperparameters
params = {'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 1000, 'multi_class': 'auto', 'verbose': 0, "random_state": 888}

## train the model
lr = LogisticRegression(**params) ## keyword arguments
lr.fit(X_train, y_train)

c:\Users\Rizwan\Desktop\Courses\MLOps\MLFLOW\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=888)

In [29]:
X_test

array([[6.1, 2.8, 4.7, 1.2],
       [5.7, 3.8, 1.7, 0.3],
       [7.7, 2.6, 6.9, 2.3],
       [6. , 2.9, 4.5, 1.5],
       [6.8, 2.8, 4.8, 1.4],
       [5.4, 3.4, 1.5, 0.4],
       [5.6, 2.9, 3.6, 1.3],
       [6.9, 3.1, 5.1, 2.3],
       [6.2, 2.2, 4.5, 1.5],
       [5.8, 2.7, 3.9, 1.2],
       [6.5, 3.2, 5.1, 2. ],
       [4.8, 3. , 1.4, 0.1],
       [5.5, 3.5, 1.3, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.1, 3.8, 1.5, 0.3],
       [6.3, 3.3, 4.7, 1.6],
       [6.5, 3. , 5.8, 2.2],
       [5.6, 2.5, 3.9, 1.1],
       [5.7, 2.8, 4.5, 1.3],
       [6.4, 2.8, 5.6, 2.2],
       [4.7, 3.2, 1.6, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [5. , 3.4, 1.6, 0.4],
       [6.4, 2.8, 5.6, 2.1],
       [7.9, 3.8, 6.4, 2. ],
       [6.7, 3. , 5.2, 2.3],
       [6.7, 2.5, 5.8, 1.8],
       [6.8, 3.2, 5.9, 2.3],
       [4.8, 3. , 1.4, 0.3],
       [4.8, 3.1, 1.6, 0.2]])

In [30]:
## Predict on the test set
y_pred = lr.predict(X_test)
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [31]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [32]:
## mlflow tracking
import mlflow.sklearn
import mlflow.sklearn


mlflow.set_tracking_uri("http://localhost:5000")

## mlflow experiment
mlflow.set_experiment("MLFLOW QUICKSTART")

## start an mlflow run
with mlflow.start_run():
    ## log the model parameters
    mlflow.log_params(params)

    ## log the accuracy metric
    mlflow.log_metric("accuracy", accuracy)

    ## set a tag that we can use to remind ourselves what this run is about
    mlflow.set_tag("Training Info", "Basic LR model for iris dataset")

    ## infer the model signature- input schema and output schema
    signature =infer_signature(X_train,lr.predict(X_train))

    ## log the model
    model_info = mlflow.sklearn.log_model(sk_model=lr,
                                          artifact_path="iris_model",
                                          signature=signature,
                                          input_example=X_train,
                                          registered_model_name="tracking_quickstart")



Registered model 'tracking_quickstart' already exists. Creating a new version of this model...
2025/06/04 16:49:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_quickstart, version 2


🏃 View run industrious-vole-383 at: http://localhost:5000/#/experiments/347628599959393078/runs/bf0b79ef96d84fc9afccc35556bf72e6
🧪 View experiment at: http://localhost:5000/#/experiments/347628599959393078


Created version '2' of model 'tracking_quickstart'.


In [33]:
model_info.model_uri

'runs:/bf0b79ef96d84fc9afccc35556bf72e6/iris_model'

### Inferencing and validating model


In [34]:
import mlflow
from mlflow.models import validate_serving_input

model_uri = model_info.model_uri

serving_payload = """{
  "inputs": [
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      4.8,
      3.4,
      1.6,
      0.2
    ],
    [
      4.4,
      3.2,
      1.3,
      0.2
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ],
    [
      6.4,
      3.2,
      4.5,
      1.5
    ],
    [
      5.2,
      3.5,
      1.5,
      0.2
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      5.2,
      4.1,
      1.5,
      0.1
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      5.5,
      2.4,
      3.7,
      1.0
    ],
    [
      6.3,
      2.8,
      5.1,
      1.5
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      7.2,
      3.6,
      6.1,
      2.5
    ],
    [
      5.7,
      2.9,
      4.2,
      1.3
    ],
    [
      7.6,
      3.0,
      6.6,
      2.1
    ],
    [
      5.6,
      3.0,
      4.5,
      1.5
    ],
    [
      5.1,
      3.5,
      1.4,
      0.2
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      5.2,
      3.4,
      1.4,
      0.2
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      6.3,
      2.7,
      4.9,
      1.8
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      5.6,
      2.7,
      4.2,
      1.3
    ],
    [
      5.1,
      3.4,
      1.5,
      0.2
    ],
    [
      5.7,
      3.0,
      4.2,
      1.2
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.0,
      2.2,
      4.0,
      1.0
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      5.5,
      2.3,
      4.0,
      1.3
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      6.4,
      2.7,
      5.3,
      1.9
    ],
    [
      5.0,
      3.3,
      1.4,
      0.2
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ],
    [
      6.7,
      3.0,
      5.0,
      1.7
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ],
    [
      5.9,
      3.2,
      4.8,
      1.8
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      6.0,
      2.7,
      5.1,
      1.6
    ],
    [
      6.1,
      2.6,
      5.6,
      1.4
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      4.4,
      2.9,
      1.4,
      0.2
    ],
    [
      4.3,
      3.0,
      1.1,
      0.1
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      5.1,
      3.5,
      1.4,
      0.3
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      4.7,
      3.2,
      1.3,
      0.2
    ],
    [
      6.1,
      2.9,
      4.7,
      1.4
    ],
    [
      6.5,
      2.8,
      4.6,
      1.5
    ],
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      7.0,
      3.2,
      4.7,
      1.4
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      6.1,
      3.0,
      4.6,
      1.4
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      6.6,
      2.9,
      4.6,
      1.3
    ],
    [
      5.5,
      2.6,
      4.4,
      1.2
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      5.6,
      3.0,
      4.1,
      1.3
    ],
    [
      7.3,
      2.9,
      6.3,
      1.8
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      5.1,
      3.7,
      1.5,
      0.4
    ],
    [
      4.9,
      2.4,
      3.3,
      1.0
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      6.7,
      3.1,
      5.6,
      2.4
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      6.9,
      3.1,
      4.9,
      1.5
    ],
    [
      7.4,
      2.8,
      6.1,
      1.9
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      6.3,
      2.3,
      4.4,
      1.3
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ],
    [
      5.4,
      3.4,
      1.7,
      0.2
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      5.8,
      4.0,
      1.2,
      0.2
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      7.1,
      3.0,
      5.9,
      2.1
    ]
  ]
}"""


# validate the serving payload works on the model
validate_serving_input (model_uri, serving_payload)


array([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2,
       1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 2, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 1,
       0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2,
       1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2,
       1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

## load the model back for prediction as a generic python function model



In [35]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)

iris_features_name = datasets.load_iris().feature_names
results = pd.DataFrame(X_test, columns=iris_features_name)
results["actual_Class"] = y_test
results["predicted_Class"] = predictions

In [36]:
results[:5]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_Class,predicted_Class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
4,6.8,2.8,4.8,1.4,1,1


### Model Registry 
#### The MLFLOW model registry component is a centralized model store, set of APIs, and UI  to collaboratively manage the full lifecycle of an MLflow model. it provides model lineage (which MLflow experiment and run produced the model), mode versioning, model aliasing, model tagging, and annotations.

In [37]:
## mlflow experiment
mlflow.set_experiment("MLFLOW QUICKSTART")

## start an mlflow run
with mlflow.start_run():
    ## log the model parameters
    mlflow.log_params(params)

    ## log the accuracy metric
    mlflow.log_metric("accuracy", 1.0)

    ## set a tag that we can use to remind ourselves what this run is about
    mlflow.set_tag("Training Info2", "Basic LR model for iris dataset")

    ## infer the model signature- input schema and output schema
    signature =infer_signature(X_train,lr.predict(X_train))

    ## log the model
    model_info = mlflow.sklearn.log_model(sk_model=lr,
                                          artifact_path="iris_model",
                                          signature=signature,
                                          input_example=X_train

                                          )

🏃 View run hilarious-cod-495 at: http://localhost:5000/#/experiments/347628599959393078/runs/034dc14ff4e049c1a813284a5042646c
🧪 View experiment at: http://localhost:5000/#/experiments/347628599959393078


In [38]:
## inferencing from the model from the model registry
import mlflow.sklearn
model_name = "tracking_quickstart"
model_version = "latest"  # or specify a version number like "1"
model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)
model

LogisticRegression(max_iter=1000, multi_class='auto', random_state=888)

In [39]:
model_uri

'models:/tracking_quickstart/latest'

In [40]:
y_pred_new = model.predict(X_test)
y_pred_new

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])